## Let's review news from Python 3.4
### Enum supported

With 2 ways to create them.
See https://docs.python.org/3/library/enum.html.

In [1]:
from enum import Enum
class Color(Enum):
    Red = 1
    Green = 2

def print_color(c: Color):
    print(f"{c.name} is {c.value}")

print_color(Color.Red)
print([c for c in Color])

Red is 1
[<Color.Red: 1>, <Color.Green: 2>]


In [2]:
from enum import Enum
Colour = Enum('Colour', 'red green') # only provides integers, which is fine

def print_colour(c: Colour):
    print(f"{c.name} is {c.value}")

print_colour(Colour.red) # note: auto-completion is not working for values
print([c for c in Colour])

red is 1
[<Colour.red: 1>, <Colour.green: 2>]


In [3]:
# alternatively, but different
from typing import Union
Couleur = Union['rouge', 'vert']

def affiche_couleur(c: Couleur):
    print(c)
    
affiche_couleur('rouge')

rouge


## Let's review news from Python 3.5

## `*` Iterable unpacking operator
Allows to extract all elements from a list/iterable

See https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-448

In [4]:
l = [1, 2, 3, 4, 5, 6]
print(l)
print(*l)
a, b, c, _, _, _ = l
print(a, b, c)

[1, 2, 3, 4, 5, 6]
1 2 3 4 5 6
1 2 3


## `**` Dictionary unpacking operator
Same as `*`, but for dictionaries.

See https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-448

In [5]:
d1 = {"key1": "value1", "key2": "value2"}
print(d1)
print(*d1)
# print(**d1) : error, print cannot take so many arguments

d2 = {"key3": "value3", "key3": "value3"}

d11 = {**d1}
print(d11)

d12 = {**d1, **d2}
print(d12)

# print(d1 + d2): + does not know how to add dictionaries

def hello(*args, **kwargs):
    # here, * and ** does reverse-unpackaging!
    print(args) # args: non-named arguments
    print(kwargs) # kwargs: KeyWord arguments
    internal = {"key": "value"}
    new = {**kwargs, **internal}
    print(new)
    
hello(1, 2, 3, x=1, y=3)

{'key1': 'value1', 'key2': 'value2'}
key1 key2
{'key1': 'value1', 'key2': 'value2'}
{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}
(1, 2, 3)
{'x': 1, 'y': 3}
{'x': 1, 'y': 3, 'key': 'value'}


## async/await keywords

Asyncio is only suitable for IOs.

`async` allows to define a function that is `await`able.

`await` allows to wait for an `async` function to return.
It allows the event_loop to pause this task, and resume it when a specific event is received.

If a function bogs the CPU, the event_loop running on the same thread will also be stopped to run by the OS.
This is why asyncio should only be used for IOs, and not CPU intensive tasks.

Warning: some libraries might look like only doing IO, but might be actively blocking (ex: python's `requsets`).
Consequence: libraries need to be fully designed with async in mind to work properly.

Following example modified from https://docs.python.org/3/whatsnew/3.5.html#pep-492-coroutines-with-async-and-await-syntax.

In [6]:
import asyncio
import random


async def http_get(domain):
    reader, writer = await asyncio.open_connection(domain, 80) # await = task is paused, will be waked up by an event
    # Here, event was received, and function has been resumed
    # at this point, a connection has been made, nothing has been sent
    
    # We now send a request
    writer.write(b'\r\n'.join([
        b'GET / HTTP/1.1',
        b'Host: %b' % domain.encode('latin-1'),
        b'Connection: close',
        b'', b''
    ]))
    
    # the response is not there yet.
    # We read the response with a reader, when it comes (streamed).
    # Paused while it is in transit
    async for line in reader:
        await asyncio.sleep(random.randint(3, 9) / 100.0) # increase the delay between bits of the stream
        print(f"{domain} >>>", line)

    writer.close()

loop = asyncio.get_event_loop()
# jupyter already is in an asyncio context (Tornado), so cannot create a new event_loop inside
# See https://stackoverflow.com/questions/47518874/how-do-i-run-python-asyncio-code-in-a-jupyter-notebook
t1 = loop.create_task(http_get("amadeus.com"))
t2 = loop.create_task(http_get("google.co.jp"))
t3 = loop.create_task(http_get("google.de"))
print("after") # probably printed before, as the request is not there yet from the network

# Explicitly wait for all tasks to but finished before running the next cell
await asyncio.gather(t1, t2, t3)

after
google.co.jp >>> b'HTTP/1.1 301 Moved Permanently\r\n'
amadeus.com >>> b'HTTP/1.1 301 Moved Permanently\r\n'
google.de >>> b'HTTP/1.1 301 Moved Permanently\r\n'
google.co.jp >>> b'Location: http://www.google.co.jp/\r\n'
amadeus.com >>> b'Location: https://amadeus.com/\r\n'
google.de >>> b'Location: http://www.google.de/\r\n'
google.co.jp >>> b'Content-Type: text/html; charset=UTF-8\r\n'
amadeus.com >>> b'Content-Length: 0\r\n'
google.de >>> b'Content-Type: text/html; charset=UTF-8\r\n'
google.co.jp >>> b'Date: Fri, 30 Oct 2020 16:40:36 GMT\r\n'
amadeus.com >>> b'Connection: close\r\n'
google.de >>> b'Date: Fri, 30 Oct 2020 16:40:36 GMT\r\n'
google.co.jp >>> b'Expires: Sun, 29 Nov 2020 16:40:36 GMT\r\n'
amadeus.com >>> b'\r\n'
google.de >>> b'Expires: Sun, 29 Nov 2020 16:40:36 GMT\r\n'
google.co.jp >>> b'Cache-Control: public, max-age=2592000\r\n'
google.de >>> b'Cache-Control: public, max-age=2592000\r\n'
google.co.jp >>> b'Server: gws\r\n'
google.de >>> b'Server: gws\r\n'
google

[None, None, None]

## Let's review news from Python 3.6

### String literales (aka f-string)

See https://www.python.org/dev/peps/pep-0498

In [7]:
a = 3
print(f"Hello number {a}")

Hello number 3


### underscore to separate big numbers
See https://www.python.org/dev/peps/pep-0515

In [8]:
print(1_000)

1000


### Async/await can be used with generators (yield): async generators
First, review of a normal generator (here calculating the fibonacci numbers)

In [9]:
import time

def fibonacci(limit: int):
    f0 = 0
    f1 = 1
    fn_1 = f1
    fn_2 = f0
    fn = f0
    while fn < limit:
        yield fn
        fn = fn_1 + fn_2
        fn_2 = fn_1
        fn_1 = fn
        time.sleep(random.randint(3, 9) / 10.0) # Simulate blocking IO
        
        
        
sequence = fibonacci(100)
        
print(sequence.__next__())
print(sequence.__next__())
print(sequence.__next__())
print(sequence.__next__())
print(sequence.__next__())

print("with a loop")
for f in fibonacci(100):
    print(f)

0
1
2
3
5
with a loop
0
1
2
3
5
8
13
21
34
55
89


Now, same, but asynchronously.

Not that this also allows `async for` loops using those generators.

See https://www.python.org/dev/peps/pep-0525.

In [10]:
async def fibonacci(limit: int):
    f0 = 0
    f1 = 1
    fn_1 = f1
    fn_2 = f0
    fn = f0
    while fn < limit:
        yield fn
        fn = fn_1 + fn_2
        fn_2 = fn_1
        fn_1 = fn
        await asyncio.sleep(random.randint(3, 9) / 10.0) # Simulate async IO

        
async def my_task():
    sequence = fibonacci(100)
        
    print(await sequence.__anext__())
    print(await sequence.__anext__())
    print(await sequence.__anext__())
    print(await sequence.__anext__())
    print(await sequence.__anext__())

    print("with a loop")
    async for f in fibonacci(100):
        print(f)
        
    # also works in list comprehension
    f_numbers = [n async for n in fibonacci(100)]
    print(f_numbers)
        
loop = asyncio.get_event_loop()
t1 = loop.create_task(my_task())
await asyncio.gather(t1)

0
1
2
3
5
with a loop
0
1
2
3
5
8
13
21
34
55
89
[0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]


[None]

### Know all subclasses that exist in the code
`__init_subclass__` is called when a subclass is defined (not need to instanciate it).
See https://www.python.org/dev/peps/pep-0487.

In [11]:
class PluginBase:
    subclasses = []

    def __init_subclass__(cls, **kwargs): # called when the subclass is defined, not when instanciated
        super().__init_subclass__(**kwargs)
        cls.subclasses.append(cls)

class Plugin1(PluginBase):
    pass

class Plugin2(PluginBase):
    pass

# Try to un-comment me!
# class Plugin2(PluginBase):
#    pass

print(Plugin1.subclasses) # No need to create an instance to know about all known plugins to Python

[<class '__main__.Plugin1'>, <class '__main__.Plugin2'>]


This allows to put constraints on inheritating classes. For example a "closed class":

In [12]:
class ClosedPluginBase:
    subclasses = []

    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        if cls.__name__ not in ["Plugin1", "Plugin4"]:
            raise RuntimeError("Disallowed to create such an instance")
        cls.subclasses.append(cls)

class Plugin1(ClosedPluginBase):
    pass

class Plugin2(ClosedPluginBase): # Exception raised here
    pass

RuntimeError: Disallowed to create such an instance

## Let's review news from Python 3.7

### Postponed annotations checks
Annotations can only used pre-defined types, but not types that are either currently defined, or defined later in the file.

See https://www.python.org/dev/peps/pep-0563

In [13]:
class Hello:
    def __init__(self, value: int):
        self.value = value

    def make(value: int) -> Hello:
        return Hello(value)
print(Hello.make(1))

NameError: name 'Hello' is not defined

One way to fix this, is to use 'string' annotations:

In [14]:
class Hello:
    def __init__(self, value: int):
        self.value = value

    def make(value: int) -> "Hello":
        return Hello(value)
print(Hello.make(1))

Now, and until 3.10, we can use a future import to fix this:

In [15]:
from __future__ import annotations

class Again:
    def __init__(self, value: int):
        self.value = value

    def make(value: int) -> Again:
        return Again(value)
print(Again.make(1))

### New breakpoint() function
It's now possible to force being dropped in the debugger by using `breakpoint()` in the code.

See https://www.python.org/dev/peps/pep-0553

In [16]:
a = 2
a += 3
breakpoint()
print(a)

--Return--
None
> <ipython-input-16-63d2074c7f4a>(3)<module>()
      1 a = 2
      2 a += 3
----> 3 breakpoint()
      4 print(a)

ipdb> print(a)
5
ipdb> quit


BdbQuit: 

It's an alternative to running your script with `python3 -m pdb myscript.py`.
See https://docs.python.org/3/library/pdb.html for more info

### Timing can now use nanoseconds resolution
See https://www.python.org/dev/peps/pep-0564.

In [17]:
import time
start = time.time_ns()
print("hello")
end = time.time_ns()
print(f"Duration: {end - start} ns")

hello
Duration: 185558 ns


### ContextVars allow defining vars with different values for different contexts
Allows each asyncio.Task to store their own context when defined, and avoid changing variables used by other tasks.

See https://www.python.org/dev/peps/pep-0567/.

In [18]:
from contextvars import ContextVar, copy_context
import contextvars

variable1 = ContextVar('variable1')
variable1.set('spam')

def main():
    variable1.set('ham')

ctx = copy_context() # deep copy of the current context (the default one)

# Any changes that the 'main' function makes to 'var'
# will be contained in 'ctx'.
ctx.run(main)

# The 'main()' function was run in the 'ctx' context,
# so changes to 'var' are contained in it:
print(ctx[variable1] == 'ham')

# However, outside of 'ctx', 'var' is still set to 'spam':
print(variable1.get() == 'spam')

# If we run main directly, it is executed in the default context, modifying the current context's var
main()
print(variable1.get() == 'ham')

# We can define a new context from here:
main_context = contextvars.Context() 
print((variable1 in main_context) is False)

True
True
True
True


### Dataclasses
Allows to make writing classes with fewer lines of code (and auto-generate some methods) aka like Lombok in Java.

It allows to avoid to unit-test `__eq__` and `__repr__` and `__hash__`.

Makes unit-testing easier (thanks to `__eq__`).

See https://www.python.org/dev/peps/pep-0557.

In [19]:
class PointLegacy:
    def __init__(self, x: float, y:float, z:float = 0.0):
        self.x = x
        self.y = y
        self.z = z
    
    def __eq__(self, o: other):
        return (self.x == o.x) and (self.y == o.y) and (self.z == o.z)
    
    def __repr__(self):
        return f"PointLegacy(x={self.x}, y={self.y}, z={self.z})"
    
    # def __hash__ too

p1 = PointLegacy(1.5, 2.5)
print(p1)

PointLegacy(x=1.5, y=2.5, z=0.0)


In [20]:
from dataclasses import dataclass

@dataclass
class Point:
    x: float
    y: float
    z: float = 0.0

p = Point(1.5, 2.5)
print(p)

Point(x=1.5, y=2.5, z=0.0)


What happens with composite classes?

In [21]:
from dataclasses import dataclass

class Color:
    def __init__(self, value: str):
        self.value = value

@dataclass
class Point:
    x: float
    y: float
        
@dataclass
class Triangle:
    a: Point
    b: Point
    c: Point
    color: Color
        
c1 = Triangle(Point(1, 2), Point(2, 3), Point(3, 4), Color("blue"))
print(c1)
c2 = Triangle(Point(1, 2), Point(2, 3), Point(3, 4), Color("blue"))
print(c1 == c2) # because the color inside are different instances, and don't support __eq__

Triangle(a=Point(x=1, y=2), b=Point(x=2, y=3), c=Point(x=3, y=4), color=<__main__.Color object at 0x7f8edc48fa00>)
False


## Let's review news from Python 3.8
### "walrus" operator
Allows to avoid to test and assign twice.
See https://www.python.org/dev/peps/pep-0572.

In [22]:
s = "1,2,3"
if len(s.split(',')) > 2:
    elements = s.split(',')
    print(elements)
    print(elements[2])

['1', '2', '3']
3


In [23]:
s = "1,2,3"
if (elements := s.split(',')) and len(elements) > 2:
    print(elements)
    print(elements[2])

['1', '2', '3']
3


### Positional-only paramaters

Using `/` allows to disable clients from naming functions parameters.

Can be useful for functions of 1 parameter, or when parameters names are useless (ex: `div(a,b)` or `len(obj)`).
It also allows hiding the name to clients, to avoid breaking clients when parameters are renamed.

Beware, can be dangerous.

See https://www.python.org/dev/peps/pep-0570.

In [24]:
def show(a: string, /, debug: bool):
    if debug:
        print(f"Debug: {a}")
    else:
        print(a)
show("hello", False)
show("again", debug=True)
show(a="ciao", debug=False)

hello
Debug: again


TypeError: show() got some positional-only arguments passed as keyword arguments: 'a'

## Let's review news from Python 3.9¶

> Note:
>
> How to run python 3.9 in jupyter via docker inside python:
>
> Ex in bash: docker run -- python:3.9 python -c 'd1 = {"spam": 1} ; d2 = {3665: 2} ; d1 |= d2 ; print(d1)'
>
> Equivalent in python:
>
> import docker
>
> client = docker.from_env()
>
> client.containers.run("python:3.9", "python -c 'd1 = {\"spam\": 1} ; d2 = {3665: 2} ; d1 |= d2 ; print(d1)'")


### New dictionary operators

Concatenate (`|`) and append (`|=`) (equivalent to `+` and `+=` for strings).

See https://www.python.org/dev/peps/pep-0584.

In [25]:
d1 = {"key1": "value1", "key2": "value2"}
d2 = {"key3": "value3", "key3": "value3"}

d12 = {**d1, **d2}
print(d12)
# d12 = d1 | d2

# d1 |= d2
print(d1 == d12) # True

{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}
False


### standard types allow generics without using typing module

Allows saying `dict[str, int]` instead of `Dict[str, int]` (same for `List`, etc.).

See https://www.python.org/dev/peps/pep-0585/.

In [26]:
from typing import Dict
d: Dict[str, int] = { "hello": 5}

In [27]:
from __future__ import annotations # only useful for 3.7 and 3.8
d2: dict[str, int] = { "again": 3 }

### removeprefix & removesuffix for strings

See https://www.python.org/dev/peps/pep-0616/.

In [28]:
test_func_name = "test_hello_there"
if test_func_name.startswith("test_"):
    print(test_func_name[5:])
else:
    print(test_func_name)
    
# OR, but everywhere in the string:

print(test_func_name.replace("test_", ""))

hello_there
hello_there


In [29]:
test_func_name = "test_hello_there"
print(test_func_name.removeprefix("test_"))

AttributeError: 'str' object has no attribute 'removeprefix'

### Being able to use any metadata for annotated types

Being able to use `Annotated` to provide more annotation to a specific type.

See https://www.python.org/dev/peps/pep-0593/.

In [ ]:
from typing import TypeVar, Annotated, List, Tuple, MaxLen, ValueRange
T = TypeVar('T')
Vec = Annotated[List[Tuple[T, T]], MaxLen(10)]
V = Vec[int]

V == Annotated[List[Tuple[int, int]], MaxLen(10)]

AGE = Annotated[int, ValueRange(0, 150)]

age: AGE = 300 # not enforced by python, as usual